# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weatherpy_df = pd.read_csv("../output_data/cities_data.csv")
weatherpy_df

,City Name,Country,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloudiness,Date
0,ushuaia,AR,-68.30,-54.80,44.60,56,20.80,40,1602884790
1,atuona,PF,-139.03,-9.80,78.58,74,18.19,0,1602884790
2,chicama,PE,-79.15,-7.84,69.39,65,7.45,7,1602884791
3,mehamn,NO,27.85,71.04,30.20,92,6.93,40,1602884791
4,rocha,UY,-54.33,-34.48,59.00,81,11.52,39,1602884791
...,...,...,...,...,...,...,...,...,...
552,beckley,US,-81.19,37.78,53.60,40,5.82,1,1602884866
553,tandalti,SD,31.87,13.02,91.02,18,2.71,36,1602884867
554,fuxin,CN,121.66,42.02,45.30,51,4.97,0,1602884867
555,maneadero,MX,-116.57,31.72,73.00,83,1.99,0,1602884867


In [33]:
weatherpy_df['Humidity'].max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

locations = weatherpy_df[["Latitude", "Longitude"]]
humidity = weatherpy_df["Humidity"]

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_city_df = weatherpy_df.loc[(weatherpy_df["Temperature"] >= 70) & (weatherpy_df["Temperature"] <= 80) & \
                                  (weatherpy_df["Wind Speed"] <= 10) & (weatherpy_df["Cloudiness"] == 0)].dropna()
narrow_city_df

,City Name,Country,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloudiness,Date
19,buin,CL,-70.75,-33.73,75.20,44,5.82,0,1602884687
172,kutum,SD,24.67,14.20,77.72,18,9.44,0,1602884812
222,salalah,OM,54.09,17.02,75.88,54,0.67,0,1602884818
260,kolaras,IN,77.60,25.23,75.40,48,7.49,0,1602884823
330,tura,IN,90.22,25.52,72.90,93,3.56,0,1602884834
376,coruripe,BR,-36.18,-10.13,74.80,81,8.52,0,1602884840
422,bahraich,IN,81.60,27.58,73.74,70,3.13,0,1602884847
469,vallenar,CL,-70.76,-28.57,77.54,24,5.97,0,1602884854
532,kununurra,AU,128.73,-15.77,77.00,100,5.82,0,1602884652
541,buraydah,SA,43.98,26.33,73.40,25,1.12,0,1602884865


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
import json

In [26]:
hotel_df = narrow_city_df
hotel_df["Hotel Name"] = ""

In [29]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    response = response.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City Name,Country,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
19,buin,CL,-70.75,-33.73,75.20,44,5.82,0,1602884687,Paihuen
172,kutum,SD,24.67,14.20,77.72,18,9.44,0,1602884812,Dibbo areaمنطقة دبو
222,salalah,OM,54.09,17.02,75.88,54,0.67,0,1602884818,HAMDAN PLAZA HOTEL
260,kolaras,IN,77.60,25.23,75.40,48,7.49,0,1602884823,Shiv Shakti Garden (Gayatri Colony)
330,tura,IN,90.22,25.52,72.90,93,3.56,0,1602884834,Hotel RIKMAN Continental
376,coruripe,BR,-36.18,-10.13,74.80,81,8.52,0,1602884840,Pousada e Hotel Litoral Sul
422,bahraich,IN,81.60,27.58,73.74,70,3.13,0,1602884847,Narain Guest House
469,vallenar,CL,-70.76,-28.57,77.54,24,5.97,0,1602884854,Humacao Bed & Breakfast
532,kununurra,AU,128.73,-15.77,77.00,100,5.82,0,1602884652,Hotel Kununurra
541,buraydah,SA,43.98,26.33,73.40,25,1.12,0,1602884865,Mövenpick Hotel Qassim


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [32]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))